In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64


## Build a scikit-learn model

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data  
Y = iris.target
# print(iris.DESCR)

In [3]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, Y)

GaussianNB(priors=None)

In [4]:
#clf.__dict__

## Generate SQL Code from the Model

In [5]:

def test_ws_sql_gen(pickle_data):
    WS_URL="http://192.168.88.88:1888/model"
    b64_data = base64.b64encode(pickle_data)
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;


In [6]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)
print(lSQL[0:2000])

WITH "RawScores" AS 
(SELECT "ADS"."KEY" AS "KEY", CAST(NULL AS FLOAT(53)) AS "LogProba_0", CAST(NULL AS FLOAT(53)) AS "Proba_0", -1.09861228867 + (-0.5 * -0.267793442091 - (0.5 * ("ADS"."Feature_0" - 5.006) * ("ADS"."Feature_0" - 5.006)) / 0.121764003092) + (-0.5 * -0.112109357731 - (0.5 * ("ADS"."Feature_1" - 3.418) * ("ADS"."Feature_1" - 3.418)) / 0.142276003092) + (-0.5 * -1.68535226038 - (0.5 * ("ADS"."Feature_2" - 1.464) * ("ADS"."Feature_2" - 1.464)) / 0.0295040030924) + (-0.5 * -2.64826613862 - (0.5 * ("ADS"."Feature_3" - 0.244) * ("ADS"."Feature_3" - 0.244)) / 0.0112640030924) AS "Score_0", CAST(NULL AS FLOAT(53)) AS "LogProba_1", CAST(NULL AS FLOAT(53)) AS "Proba_1", -1.09861228867 + (-0.5 * 0.495040594659 - (0.5 * ("ADS"."Feature_0" - 5.936) * ("ADS"."Feature_0" - 5.936)) / 0.261104003092) + (-0.5 * -0.500335172182 - (0.5 * ("ADS"."Feature_1" - 2.77) * ("ADS"."Feature_1" - 2.77)) / 0.0965000030924) + (-0.5 * 0.30725034869 - (0.5 * ("ADS"."Feature_2" - 4.26) * ("ADS"."Feature

## Execute the SQL Code

In [7]:
# save the dataset in a database table

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(X);
lTable.columns = ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3']
lTable['KEY'] = range(lTable.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)


In [8]:
sql_output = pd.read_sql(lSQL , conn);


In [9]:
sql_output.sample(12)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
135,135,-581.027263,-26.970110,-3.837318,2.135952e-251,8.985786e-11,1.000000e+00,-5.771899e+02,-23.132792,-8.985790e-11,2
82,82,-144.275450,-1.105071,-11.600841,6.635631e-63,9.999723e-01,2.765239e-05,-1.431704e+02,-0.000028,-1.049580e+01,1
131,131,-583.416359,-29.929205,-7.991009,1.247227e-250,2.967305e-10,1.000000e+00,-5.754254e+02,-21.938197,-2.967304e-10,2
130,130,-509.578491,-16.767758,-2.853796,8.555803e-221,9.062384e-07,9.999991e-01,-5.067247e+02,-13.913963,-9.062389e-07,2
1,1,0.442626,-38.307886,-56.290206,1.000000e+00,1.482062e-17,2.297440e-25,0.000000e+00,-38.750512,-5.673283e+01,0
100,100,-580.273435,-27.224121,-3.744740,4.137795e-251,6.353810e-11,1.000000e+00,-5.765287e+02,-23.479381,-6.353806e-11,2
122,122,-631.371544,-26.183870,-5.183850,1.122379e-272,7.582404e-10,1.000000e+00,-6.261877e+02,-21.000021,-7.582404e-10,2
38,38,-1.405606,-42.115238,-60.143319,1.000000e+00,2.089448e-18,3.094109e-26,0.000000e+00,-40.709632,-5.873771e+01,0
51,51,-233.118091,-2.436088,-5.283208,6.188548e-101,9.451696e-01,5.483036e-02,-2.307384e+02,-0.056391,-2.903511e+00,1
35,35,-0.176078,-41.349871,-58.973029,1.000000e+00,1.313557e-18,2.916143e-26,0.000000e+00,-41.173793,-5.879695e+01,0


In [10]:
sql_output.Decision.value_counts()

2    50
1    50
0    50
Name: Decision, dtype: int64

## Scikit-learn Prediction

In [11]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
141,141,NaN,NaN,NaN,1.610931e-184,4.677981e-07,9.999995e-01,-423.198845,-1.457523e+01,-4.677982e-07,2
9,9,NaN,NaN,NaN,1.000000e+00,3.203442e-18,1.129895e-25,0.000000,-4.028231e+01,-5.744250e+01,0
64,64,NaN,NaN,NaN,1.725091e-55,9.999754e-01,2.461264e-05,-126.096900,-2.461294e-05,-1.061225e+01,1
7,7,NaN,NaN,NaN,1.000000e+00,6.576153e-18,2.790210e-25,0.000000,-3.956308e+01,-5.653851e+01,0
84,84,NaN,NaN,NaN,3.716474e-98,9.924766e-01,7.523362e-03,-224.340564,-7.551805e-03,-4.889742e+00,1
102,102,NaN,NaN,NaN,1.049417e-218,1.679154e-07,9.999998e-01,-501.915316,-1.559981e+01,-1.679154e-07,2
30,30,NaN,NaN,NaN,1.000000e+00,1.050324e-16,2.326775e-24,0.000000,-3.679226e+01,-5.441756e+01,0
145,145,NaN,NaN,NaN,6.477323e-189,5.855080e-07,9.999994e-01,-433.320275,-1.435079e+01,-5.855081e-07,2
122,122,NaN,NaN,NaN,1.122379e-272,7.582404e-10,1.000000e+00,-626.187694,-2.100002e+01,-7.582406e-10,2
93,93,NaN,NaN,NaN,6.269125e-35,9.999998e-01,2.024879e-07,-78.754841,-2.024879e-07,-1.541259e+01,1


## Comparing the SQL and Scikit-learn Predictions

In [12]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [13]:
sql_skl_join.sample(12)

,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
122,122,NaN,NaN,NaN,1.122379e-272,7.582404e-10,1.000000e+00,-626.187694,-21.000021,-7.582406e-10,...,-631.371544,-26.183870,-5.183850,1.122379e-272,7.582404e-10,1.000000e+00,-626.187694,-21.000021,-7.582404e-10,2
90,90,NaN,NaN,NaN,3.927107e-82,9.998737e-01,1.263203e-04,-187.444075,-0.000126,-8.976690e+00,...,-188.747817,-1.303869,-10.280432,3.927107e-82,9.998737e-01,1.263203e-04,-187.444075,-0.000126,-8.976690e+00,1
62,62,NaN,NaN,NaN,6.401655e-61,9.999940e-01,6.016325e-06,-138.601134,-0.000006,-1.202103e+01,...,-142.372819,-3.771691,-15.792719,6.401655e-61,9.999940e-01,6.016325e-06,-138.601134,-0.000006,-1.202103e+01,1
141,141,NaN,NaN,NaN,1.610931e-184,4.677981e-07,9.999995e-01,-423.198845,-14.575229,-4.677982e-07,...,-425.512466,-16.888851,-2.313622,1.610931e-184,4.677981e-07,9.999995e-01,-423.198845,-14.575229,-4.677982e-07,2
46,46,NaN,NaN,NaN,1.000000e+00,2.396423e-18,3.119092e-25,0.000000,-40.572554,-5.642709e+01,...,0.309658,-40.262896,-56.117427,1.000000e+00,2.396423e-18,3.119092e-25,0.000000,-40.572554,-5.642709e+01,0
76,76,NaN,NaN,NaN,4.950122e-113,9.128444e-01,8.715556e-02,-258.592703,-0.091190,-2.440061e+00,...,-261.218561,-2.717048,-5.065919,4.950122e-113,9.128444e-01,8.715556e-02,-258.592703,-0.091190,-2.440061e+00,1
86,86,NaN,NaN,NaN,1.057867e-111,7.992948e-01,2.007052e-01,-255.530691,-0.224026,-1.605918e+00,...,-258.368669,-3.062004,-4.443897,1.057867e-111,7.992948e-01,2.007052e-01,-255.530691,-0.224026,-1.605918e+00,1
78,78,NaN,NaN,NaN,4.197027e-100,9.864803e-01,1.351973e-02,-228.824133,-0.013612,-4.303605e+00,...,-229.971706,-1.161185,-5.451178,4.197027e-100,9.864803e-01,1.351973e-02,-228.824133,-0.013612,-4.303605e+00,1
48,48,NaN,NaN,NaN,1.000000e+00,2.704142e-18,2.864928e-25,0.000000,-40.451747,-5.651208e+01,...,0.515844,-39.935903,-55.996240,1.000000e+00,2.704142e-18,2.864928e-25,0.000000,-40.451747,-5.651208e+01,0
143,143,NaN,NaN,NaN,2.540294e-231,4.425560e-09,1.000000e+00,-530.964877,-19.235869,-4.425560e-09,...,-533.245692,-21.516685,-2.280816,2.540294e-231,4.425560e-09,1.000000e+00,-530.964877,-19.235869,-4.425560e-09,2


In [14]:
condition = (sql_skl_join.Decision_sql != sql_skl_join.Decision_skl)
sql_skl_join[condition]


,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
